In [ ]:
import os
import pandas as pd

# Path to the directory containing the CSV files
directory_path = '/data1-8tb-blk/sp2023stock/Insight-Wave/TopicModeling/result/integrationOutput/id/prachachat_ids.csv'

articles_list = {}

df = pd.read_csv(directory_path)
articles_list = df['ID'].tolist()

print(len(articles_list))


In [5]:
import os
import csv
import re
import pandas as pd
import chardet
from datetime import datetime
from bs4 import BeautifulSoup


# Detect input file
def htmlInput(file_path):
    with open(file_path, "r", encoding='utf-8') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, "html.parser")
    return soup


#Extract title
def extract_title(soup):
    title = soup.title.string.strip() if soup.title else None
    return title

#Extract body
def extract_body(soup):
    article_divs = soup.find_all("div", itemprop = "articleBody")
    articles = []
    if article_divs:
        
        for inside_div in article_divs:
            articleses = inside_div.find_all('p')
            for article in articleses:
                articles.append(article.get_text(strip=True))

        articles = [s.replace('\xa0', ' ') for s in articles]
        articles = [item for item in articles if item != '']
    else:
        articles = None
    
    return articles

#Extract tags
def extract_tags(soup):
    top_div = soup.find('div', class_='ud-post-category-title')
    bott_div = soup.find('div', class_='td-post-source-tags')

    top_tag = top_div.get_text(strip=True)

    bott_tag = []

    if bott_div:
        tag_links = bott_div.find_all('a')
        for tag_link in tag_links:
            bott_tag.append(tag_link.get_text(strip=True))

    if bott_tag and top_tag:
        tag_content = bott_tag + [top_tag]
    elif not bott_tag :
        tag_content = top_tag

    return tag_content

#Extract date
def extract_pubdate(soup):
    date_time = soup.find('time', {'class': 'entry-date'}).get('datetime')
    if date_time is not None:
        date_object = datetime.strptime(date_time, "%Y-%m-%dT%H:%M:%S%z")
        pubdate = date_object.strftime("%d/%m/%Y %H:%M:%S")
         
    else:
        pubdate = None
    
    return pubdate


#Extract title
def extract_intro(soup):
    intro_divs = soup.find_all("div", itemprop="articleBody")

    intro = None  # Default value if nothing is found
    for intro_div in intro_divs:
        intro_paragraphs = intro_div.find_all('p')
        for intro_paragraph in intro_paragraphs:
            intro_strong = intro_paragraph.find('strong')
            if intro_strong:
                intro = intro_strong.get_text(strip=True)
                break  # Exit the loop if we found what we were looking for
    
        if intro is None:
            intro = intro_paragraphs[0].get_text(strip=True)
            break  # Exit the outer loop if we found what we were looking for

    return intro

def extract_urlPic(soup):
    img_div = soup.find("div", class_="td-post-featured-image")

    if img_div is not None:
        img_tag = img_div.find("img", class_="entry-thumb")

        if img_tag is not None:
            image_link = img_tag.get("data-cfsrc")
        else:
            image_link = None
    else:
        image_link = None

    return image_link


#start using "csv_progress" (current progress of parsing)
if os.path.exists('parsing_progress.txt'):
    with open('parsing_progress.txt','r') as f:
        progress_start = int(f.readline().strip())
        start_id = progress_start - 1
else: 
    start_id = 1 
    
article_id = start_id


process = 'E:/Crimson_News/src/CrawlingCode/prachachat/progress.txt'

#start using "progress" (last progress of crwaling)
if os.path.exists(process):
    with open(process,'r') as f:
        progress_end = int(f.readline().strip())
        end_id = progress_end + 1

for article_id in range(start_id, end_id+1):
    file_path = "E:/Crimson_News/DataSet/prachachat/article/" + str(article_id) + "/index.txt"
    
    if os.path.exists(file_path):
        try:
        
            soup = htmlInput(file_path)
            title = extract_title(soup)
            intro = extract_intro(soup)
            articles = extract_body(soup)
            pubdate = extract_pubdate(soup)
            tag_content = extract_tags(soup)
            url = extract_urlPic(soup)

            if title == intro:
                intro = None
            if articles and intro == articles[0]:
                articles.pop(0)
            if title == articles[0]:
                articles.pop(0)


            data_dict = {'Title': title, 'Intro': intro, 'Article': articles, 'DateTime': pubdate, 'Tags': tag_content, 'url_picture': url}

            with open(os.path.join("E:/Crimson_News/DataSet/prachachat/article/" + str(article_id) +"/parsing.txt"), 'w', encoding="utf-8") as f:
                for key, value in data_dict.items():
                    if value is not None:
                        if key == 'Tags' and isinstance(value, list):
                            f.write(f"[::{key}::]\n")
                            for tags in value:
                                f.write(f"{tags}\n")

                        elif key == 'Article' and isinstance(value, list):
                            f.write(f"[::{key}::]\n")
                            for bodys in value:
                                if bodys == 'SPONSORED':
                                    continue
                                else:
                                    f.write(f"{bodys}\n")
                                    f.write("\n")
                        else:
                            f.write(f"[::{key}::]\n{value}\n")


            with open('parsing_progress.txt', 'w') as f:
                f.write(f'{article_id}')

            with open('parsing_number.csv', 'a', newline='') as f:
                writer = csv.writer(f)
                validation = 'Valid'
                writer.writerow([article_id, validation])

                    
        except Exception as e:
            print(f"Error processing article {article_id}: {e}")
            with open('parsing_number.csv', 'a', newline='') as f:
                writer = csv.writer(f)
                validation = 'Error'
                writer.writerow([article_id, validation])
                continue


# Write the last processed article_id to 'csv_progress.txt' after loop ends
with open('parsing_progress.txt', 'w') as f:
    f.write(str(article_id) + '\n')

Error processing article 167273: list index out of range
Error processing article 167631: list index out of range
Error processing article 167765: list index out of range
Error processing article 168004: list index out of range
Error processing article 168458: list index out of range
Error processing article 168481: list index out of range
Error processing article 168510: list index out of range
Error processing article 168542: list index out of range
Error processing article 168578: list index out of range
Error processing article 168837: list index out of range
Error processing article 169233: list index out of range
Error processing article 169435: list index out of range
Error processing article 169805: list index out of range
Error processing article 169817: list index out of range
Error processing article 169940: cannot access local variable 'tag_content' where it is not associated with a value
Error processing article 169955: list index out of range
Error processing article 1702

Error processing article 189700: list index out of range
Error processing article 189943: list index out of range
Error processing article 190140: list index out of range
Error processing article 190227: list index out of range
Error processing article 190294: list index out of range
Error processing article 190347: list index out of range
Error processing article 190663: list index out of range
Error processing article 191082: list index out of range
Error processing article 191180: list index out of range
Error processing article 191229: list index out of range
Error processing article 191258: list index out of range
Error processing article 191265: list index out of range
Error processing article 191310: list index out of range
Error processing article 191321: list index out of range
Error processing article 191540: list index out of range
Error processing article 191601: list index out of range
Error processing article 191612: list index out of range
Error processing article 191855

Error processing article 208527: list index out of range
Error processing article 208675: list index out of range
Error processing article 208688: list index out of range
Error processing article 209115: list index out of range
Error processing article 209292: list index out of range
Error processing article 209309: list index out of range
Error processing article 209348: list index out of range
Error processing article 209353: list index out of range
Error processing article 209761: list index out of range
Error processing article 210074: list index out of range
Error processing article 210205: list index out of range
Error processing article 210219: list index out of range
Error processing article 210229: list index out of range
Error processing article 210288: list index out of range
Error processing article 210653: list index out of range
Error processing article 210826: list index out of range
Error processing article 210843: list index out of range
Error processing article 211120

Error processing article 228552: list index out of range
Error processing article 228572: list index out of range
Error processing article 228738: list index out of range
Error processing article 228754: list index out of range
Error processing article 228759: list index out of range
Error processing article 229112: list index out of range
Error processing article 229133: list index out of range
Error processing article 229209: list index out of range
Error processing article 229430: list index out of range
Error processing article 229447: list index out of range
Error processing article 230018: list index out of range
Error processing article 230132: 'charmap' codec can't decode byte 0x81 in position 226: character maps to <undefined>
Error processing article 230674: list index out of range
Error processing article 230841: list index out of range
Error processing article 231048: list index out of range
Error processing article 231066: list index out of range
Error processing article 2

Error processing article 250105: list index out of range
Error processing article 250368: list index out of range
Error processing article 250768: list index out of range
Error processing article 250773: list index out of range
Error processing article 250947: list index out of range
Error processing article 250955: list index out of range
Error processing article 251096: list index out of range
Error processing article 251147: 'NoneType' object has no attribute 'get'
Error processing article 251148: 'NoneType' object has no attribute 'get'
Error processing article 251197: list index out of range
Error processing article 251309: list index out of range
Error processing article 251458: list index out of range
Error processing article 251944: list index out of range
Error processing article 251954: list index out of range
Error processing article 251960: list index out of range
Error processing article 251970: list index out of range
Error processing article 252119: list index out of ran

...

Error processing article 418787: list index out of range
Error processing article 418869: list index out of range
Error processing article 419015: list index out of range
Error processing article 419059: list index out of range
Error processing article 419110: list index out of range
Error processing article 419213: list index out of range
Error processing article 419279: list index out of range
Error processing article 419377: list index out of range
Error processing article 419501: list index out of range
Error processing article 419701: list index out of range
Error processing article 419963: list index out of range
Error processing article 421309: list index out of range
Error processing article 421466: 'charmap' codec can't decode byte 0x81 in position 253: character maps to <undefined>
Error processing article 421562: 'charmap' codec can't decode byte 0x81 in position 233: character maps to <undefined>
Error processing article 421608: list index out of range
Error processing arti

Error processing article 437814: 'charmap' codec can't decode byte 0x9d in position 260: character maps to <undefined>
Error processing article 437826: list index out of range
Error processing article 437847: list index out of range
Error processing article 437857: list index out of range
Error processing article 437908: list index out of range
Error processing article 438072: 'charmap' codec can't decode byte 0x81 in position 233: character maps to <undefined>
Error processing article 438073: 'charmap' codec can't decode byte 0x81 in position 238: character maps to <undefined>
Error processing article 438082: list index out of range
Error processing article 438171: list index out of range
Error processing article 438181: list index out of range
Error processing article 438184: list index out of range
Error processing article 438205: list index out of range
Error processing article 438237: list index out of range
Error processing article 438244: list index out of range
Error processing

Error processing article 443926: list index out of range
Error processing article 443929: list index out of range
Error processing article 443946: list index out of range
Error processing article 443951: list index out of range
Error processing article 443954: list index out of range
Error processing article 444579: list index out of range
Error processing article 444671: list index out of range
Error processing article 444877: list index out of range
Error processing article 444899: list index out of range
Error processing article 444910: list index out of range
Error processing article 444916: list index out of range
Error processing article 444930: list index out of range
Error processing article 444940: list index out of range
Error processing article 444956: list index out of range
Error processing article 444961: list index out of range
Error processing article 444965: list index out of range
Error processing article 444969: list index out of range
Error processing article 444975

Error processing article 453000: list index out of range
Error processing article 453091: list index out of range
Error processing article 453671: list index out of range
Error processing article 453830: 'charmap' codec can't decode byte 0x9e in position 226: character maps to <undefined>
Error processing article 454317: list index out of range
Error processing article 454715: list index out of range
Error processing article 454734: list index out of range
Error processing article 454945: list index out of range
Error processing article 455153: 'charmap' codec can't decode byte 0x9e in position 250: character maps to <undefined>
Error processing article 455218: 'charmap' codec can't decode byte 0x9d in position 278: character maps to <undefined>
Error processing article 455464: list index out of range
Error processing article 455479: list index out of range
Error processing article 455559: 'charmap' codec can't decode byte 0x81 in position 226: character maps to <undefined>
Error proce

Error processing article 470868: list index out of range
Error processing article 471280: list index out of range
Error processing article 471353: 'charmap' codec can't decode byte 0x9d in position 278: character maps to <undefined>
Error processing article 471463: list index out of range
Error processing article 471517: list index out of range
Error processing article 471790: 'charmap' codec can't decode byte 0x9d in position 278: character maps to <undefined>
Error processing article 471925: list index out of range
Error processing article 472205: list index out of range
Error processing article 472319: 'charmap' codec can't decode byte 0x9d in position 278: character maps to <undefined>
Error processing article 472553: list index out of range
Error processing article 472969: 'charmap' codec can't decode byte 0x81 in position 1264: character maps to <undefined>
Error processing article 473002: 'charmap' codec can't decode byte 0x9d in position 278: character maps to <undefined>
Error